# Sequence Encoding

> We define all the usefull functions to encode the TR sequence as an imput for the predictive model

In [ ]:
#| default_exp encoding

In [ ]:
# | export
import ViennaRNA as RNA
import numpy as np


In [ ]:
#| export

def base_pairing_probability_array(sequence: str, #DNA sequence
                              ):
                              
    """Calculates the probability pairing array for a given DNA sequence"""

    # Convert DNA sequence to RNA (replace T with U)
    rna_seq = sequence.replace('T', 'U')

    # Perform RNA folding using ViennaRNA library
    (propensity, ensemble_energy) = RNA.pf_fold(rna_seq)

    # Create a square array with dimensions equal to the sequence length
    size = len(sequence)
    array = np.zeros((size, size), dtype=np.float16)  # Empty array for probabilities

    # Fill the array with pairing probabilities using RNA.get_pr()
    for i in range(size):
        for j in range(size):
            array[i, j] = RNA.get_pr(i + 1, j + 1) 

    # Round probabilities to 4 decimal places
    return np.round(array, decimals=4)


In [ ]:
#| export
def base_pairing_probability_max_mean_std(sequence: str, #DNA sequence
                                       ):
    """
    Calculates summary statistics for the base pair probabilities of an DNA sequence.
    """

    # Calculate the probability pairing array using the provided function
    proba_array = base_pairing_probability_array(sequence)

    # Check if the dimensions of the pairing array and sequence match
    size_1 = len(proba_array)
    size_2 = len(sequence)
    if size_1 != size_2:
        raise ValueError("Dimensions of probability array and sequence mismatch!")

    # Create a new array to store summary statistics
    proba_array_summary = np.zeros((size_1, 3))

    # Calculate and store maximum probability for each position
    for k in range(size_1):
        proba_array_summary[k, 0] = np.max(proba_array[k])

    # Calculate and store mean probability for each position
    proba_array_summary[k, 1] = np.mean(proba_array[k])

    # Calculate and store standard deviation for each position
    proba_array_summary[k, 2] = np.std(proba_array[k])

    # Return the array of summary statistics
    return proba_array_summary


In [ ]:
# |export
def encode_mean_max_std_TR_size_indep(sequence_list:str #list of DNA sequences of TRs
                                        ):
    """
    Encodes a list DNA sequences based on base pair probability statistics.
    The DNA sequence here correspond to  20 bases of Avd + the TR sequence + 50/100/150 bases of the DGR spacer
    The encoding is independent of the TR length
    """

    encoding_list = []  # List to store encoded sequences
    Avd_seq='aagggcaggctgggaaATAA'.upper()
    DGR_spacer='tctgtgcccatcaccttcttgcatggctctgccaacgctacggcttggcgggctggcctttcctcaataggtggtcagccggttctgtcctgcttcggcgaacacgttacacggttcggcaaaacgtcgattactgaaaatggaaaggcggggccgacttc'.upper()

    for sequence in tqdm(sequence_list):
        # Define spacer lengths 
        spacer_lengths = [50, 100, 150]

        # Get transcript length
        tr_length = len(sequence)

        # Iterate through spacer lengths
        for k in spacer_lengths:
            # Construct sequence with spacer
            sequence_k = Avd_seq + sequence + DGR_spacer[:k]  # Replace with actual sequences

            # Calculate base pair probability statistics
            bppmms_k = base_pair_probability_max_mean_std(sequence_k)

            # Extract statistics for Avd region
            bppmms_avd = bppmms_k[:20, :] 

            # Extract statistics for TR region (split into 10 parts)
            bppmms_tr = bppmms_k[20:20 + tr_length, :]
            bppmms_tr_split = np.array_split(bppmms_tr, 10, axis=0)  # Split into 10 parts
            bppmms_tr_mean = np.array([np.mean(k, axis=0) for k in bppmms_tr_split])  # Mean of each split

            # Extract statistics for spacer region
            bppmms_spacer = bppmms_k[20 + tr_length:, :]  # Adjust slice based on spacer region size

            # Merge statistics for different regions
            bppmms_merged = np.concatenate([bppmms_avd, bppmms_tr_mean, bppmms_spacer], axis=0)

            # Concatenate encodings for different spacer lengths (if applicable)
            if k != spacer_lengths[0]:
                bppmms_concat = np.concatenate([bppmms_concat, bppmms_merged], axis=0)
            else:
                bppmms_concat = bppmms_merged

        # Append encoding for the current sequence
        encoding_list.append(bppmms_concat)

    # Convert to numpy array for efficient processing
    return np.array(encoding_list)
